In [1]:
import time
start_time = time.time()
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json


# Data Loading [train/test dataset ] 

- 處理 train dataset 

In [2]:
%%time
print(os.getcwd()+'/nybg2020/train/metadata.json')

train_path = os.getcwd()+'/nybg2020/train/metadata.json'

with open(train_path, "r", encoding="ISO-8859-1") as file:
    train = json.load(file)


/datas/HW/kaggleHW/nybg/nybg2020/train/metadata.json
CPU times: user 1.82 s, sys: 550 ms, total: 2.37 s
Wall time: 4.19 s


In [3]:
print(train.keys())


dict_keys(['annotations', 'categories', 'images', 'info', 'licenses', 'regions'])


In [4]:
train_cat = pd.DataFrame(train['categories'])
train_cat.columns = ['family', 'genus', 'category_id', 'categort_name']
# 科(famliy) > 屬(genus) > 種(name)
#Fabaceae 豆科 Orchidaceae 蘭科
train_cat

,family,genus,category_id,categort_name
0,Orchidaceae,Aa,0,Aa mathewsii (Rchb.f.) Schltr.
1,Orchidaceae,Aa,1,Aa paleacea (Kunth) Rchb.f.
2,Fabaceae,Abarema,2,Abarema abbottii (Rose & Leonard) Barneby & J....
3,Fabaceae,Abarema,3,Abarema acreana (J.F.Macbr.) L.Rico
4,Fabaceae,Abarema,4,Abarema adenophora (Ducke) Barneby & J.W.Grimes
...,...,...,...,...
32089,Orchidaceae,Zygopetalum,32089,Zygopetalum maxillare Lodd.
32090,Orchidaceae,Zygopetalum,32090,Zygopetalum sellowii Rchb.f.
32091,Polypodiaceae,Zygophlebia,32091,Zygophlebia mathewsii (Kunze ex Mett.) L.E.Bishop
32092,Polypodiaceae,Zygophlebia,32092,Zygophlebia sectifrons (Kunze ex Mett.) L.E.Bi...


- id 與 image id 重複 清掉其中一個

In [5]:
train_ann = pd.DataFrame(train['annotations']).drop(columns='image_id')
train_ann

,category_id,id,region_id
0,15672,354106,1
1,11524,818566,1
2,11524,750704,1
3,11467,722381,1
4,15660,382783,1
...,...,...,...
1030742,24148,856534,3
1030743,5142,541440,1
1030744,27240,927988,1
1030745,269,242745,2


In [6]:
train_img = pd.DataFrame(train['images'])
train_img

,file_name,height,id,license,width
0,images/156/72/354106.jpg,1000,354106,1,661
1,images/115/24/818566.jpg,1000,818566,1,661
2,images/115/24/750704.jpg,1000,750704,1,661
3,images/114/67/722381.jpg,1000,722381,1,661
4,images/156/60/382783.jpg,1000,382783,1,661
...,...,...,...,...,...
1030742,images/241/48/856534.jpg,1000,856534,1,682
1030743,images/051/42/541440.jpg,1000,541440,1,661
1030744,images/272/40/927988.jpg,1000,927988,1,669
1030745,images/002/69/242745.jpg,1000,242745,1,682


- 合併 train['annotations'] + train['categories']

In [7]:
train_test = train_ann.merge(train_cat, on='category_id')
train_test

,category_id,id,region_id,family,genus,categort_name
0,15672,354106,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg
1,15672,545181,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg
2,15672,449419,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg
3,15672,200223,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg
4,15672,5327,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg
...,...,...,...,...,...,...
1030742,31957,729305,3,Rutaceae,Zanthoxylum,Zanthoxylum dumosum A.Rich.
1030743,3834,574147,0,Asteraceae,Brickellia,Brickellia cuspidata A.Gray
1030744,3834,853499,0,Asteraceae,Brickellia,Brickellia cuspidata A.Gray
1030745,17312,964931,1,Marantaceae,Maranta,Maranta parvifolia Petersen


- 合併 兩張表 train_test + train_img(train['images']) by id

In [8]:
train_df = train_img.merge(train_test, on='id')
train_df

,file_name,height,id,license,width,category_id,region_id,family,genus,categort_name
0,images/156/72/354106.jpg,1000,354106,1,661,15672,1,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg
1,images/115/24/818566.jpg,1000,818566,1,661,11524,1,Lecythidaceae,Eschweilera,Eschweilera wachenheimii (Benoist) Sandwith
2,images/115/24/750704.jpg,1000,750704,1,661,11524,1,Lecythidaceae,Eschweilera,Eschweilera wachenheimii (Benoist) Sandwith
3,images/114/67/722381.jpg,1000,722381,1,661,11467,1,Lecythidaceae,Eschweilera,Eschweilera awaensis S.A.Mori & Cornejo
4,images/156/60/382783.jpg,1000,382783,1,661,15660,1,Lecythidaceae,Lecythis,Lecythis holcogyne (Sandwith) S.A.Mori
...,...,...,...,...,...,...,...,...,...,...
1030742,images/241/48/856534.jpg,1000,856534,1,682,24148,3,Potamogetonaceae,Potamogeton,Potamogeton illinoensis Morong
1030743,images/051/42/541440.jpg,1000,541440,1,661,5142,1,Lecythidaceae,Cariniana,Cariniana estrellensis (Raddi) Kuntze
1030744,images/272/40/927988.jpg,1000,927988,1,669,27240,1,Fabaceae,Senna,Senna reticulata (Willd.) H.S.Irwin & Barneby
1030745,images/002/69/242745.jpg,1000,242745,1,682,269,2,Asteraceae,Acmella,Acmella repens (Walter) Rich.


- 處理 test dataset 

In [9]:
%%time
print(os.getcwd()+'/nybg2020/test/metadata.json')
train_path =sub_path = os.getcwd()+'/nybg2020/test/metadata.json'

with open(train_path, "r", encoding="ISO-8859-1") as file:
    test = json.load(file)
    
    
test_df = pd.DataFrame(test['images'])
test_df.head()

/datas/HW/kaggleHW/nybg/nybg2020/test/metadata.json
CPU times: user 261 ms, sys: 36.2 ms, total: 297 ms
Wall time: 498 ms


,file_name,height,id,license,width
0,images/104/104891.jpg,1000,104891,1,661
1,images/018/18029.jpg,1000,18029,1,661
2,images/035/35151.jpg,1000,35151,1,661
3,images/124/124144.jpg,1000,124144,1,682
4,images/024/24649.jpg,1000,24649,1,682


In [10]:
#train_df['category_id'].value_counts()


In [11]:
sub_path =os.getcwd() +'/sample_submission.csv'
sample_submission = pd.read_csv(sub_path)
sample_submission.head()


,Id,Predicted
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [12]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_df['category_id'])
train_df['category_id_le'] = le.transform(train_df['category_id'])
class_map = dict(sorted(train_df[['category_id_le', 'category_id']].values.tolist()))


# Library loading

In [13]:
# ====================================================
# Library
# ====================================================

import sys

import gc
import os
import random
import time
from contextlib import contextmanager
from pathlib import Path

import cv2
from PIL import Image
import numpy as np
import pandas as pd
import scipy as sp

import sklearn.metrics

from functools import partial

import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, Dataset

from albumentations import Compose, Normalize, Resize
from albumentations.pytorch import ToTensorV2


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Utils

In [14]:
# ====================================================
# Utils
# ====================================================

@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')

    
def init_logger(log_file='train.log'):
    from logging import getLogger, DEBUG, FileHandler,  Formatter,  StreamHandler
    
    log_format = '%(asctime)s %(levelname)s %(message)s'
    
    stream_handler = StreamHandler()
    stream_handler.setLevel(DEBUG)
    stream_handler.setFormatter(Formatter(log_format))
    
    file_handler = FileHandler(log_file)
    file_handler.setFormatter(Formatter(log_format))
    
    logger = getLogger('Herbarium')
    logger.setLevel(DEBUG)
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    
    return logger

LOG_FILE = 'train.log'
LOGGER = init_logger(LOG_FILE)


def seed_torch(seed=777):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 777
seed_torch(SEED)

# Dataset

In [15]:
N_CLASSES = 32093


class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.df['file_name'].values[idx]
        file_path = os.getcwd()+f'/nybg2020/test/'+file_name
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image

In [16]:
# Transforms

In [17]:
HEIGHT = 128
WIDTH = 128


def get_transforms():    
    return Compose([
            Resize(HEIGHT, WIDTH),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
#     assert data in ('train', 'valid')
    
#     if data == 'train':
#         return Compose([
#             Resize(HEIGHT, WIDTH),
#             Normalize(
#                 mean=[0.485, 0.456, 0.406],
#                 std=[0.229, 0.224, 0.225],
#             ),
#             ToTensorV2(),
#         ])
    
#     elif data == 'valid':
#         return Compose([
#             Resize(HEIGHT, WIDTH),
#             Normalize(
#                 mean=[0.485, 0.456, 0.406],
#                 std=[0.229, 0.224, 0.225],
#             ),
#             ToTensorV2(),
#         ])

In [18]:
batch_size = 256

test_dataset = TestDataset(test_df, transform=get_transforms())
test_loader = DataLoader(test_dataset,num_workers=4, batch_size=batch_size, shuffle=False)


In [19]:
import torchvision.models as models

with torch.no_grad():
    model = models.resnet18(pretrained=False)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    model.fc = nn.Linear(model.fc.in_features, N_CLASSES)

    weights_path =os.getcwd() +'/fold0_best_score.pth'
    model.load_state_dict(torch.load(weights_path))


In [20]:
from tqdm import tqdm

with timer('inference'):
    
    model.to(device) 
    
    preds = np.zeros((len(test_dataset)))

    for i, images in tqdm(enumerate(test_loader)):
            
        images = images.to(device)
            
        with torch.no_grad():
            y_preds = model(images)
            
        preds[i * batch_size: (i+1) * batch_size] = y_preds.argmax(1).to('cpu').numpy()


2020-05-25 10:17:17,526 INFO [inference] start
541it [27:41,  3.07s/it]
2020-05-25 10:44:59,171 INFO [inference] done in 1662 s.


In [21]:
test_df['preds'] = preds.astype(int)
submission = sample_submission.merge(test_df.rename(columns={'id': 'Id'})[['Id', 'preds']], on='Id').drop(columns='Predicted')
submission['Predicted'] = submission['preds'].map(class_map)
submission = submission.drop(columns='preds')
submission.to_csv('0525-submission.csv', index=False)
submission.head()

,Id,Predicted
0,0,2274
1,1,1450
2,2,30649
3,3,21138
4,4,24170


# train valid split


# Model

# Train